<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/evaluate_multivers_scifact_on_climate_fever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/dwadden/multivers.git
!git clone https://github.com/allenai/scifact-evaluator.git
!pip install virtualenv
!virtualenv multivers
!source /content/multivers/bin/activate; pip install -r /content/multivers/requirements.txt

In [2]:
!python /content/multivers/script/get_checkpoint.py longformer_large_science
!python /content/multivers/script/get_checkpoint.py scifact

--2023-03-23 19:32:42--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.92.139.2, 52.218.224.137, 52.92.195.42, ...
Connecting to scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)|52.92.139.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646843693 (1.5G) [binary/octet-stream]
Saving to: ‘checkpoints/longformer_large_science.ckpt’

checkpoints/longfor 100%[===================>]   1.53G  34.6MB/s    in 48s     

2023-03-23 19:33:30 (33.0 MB/s) - ‘checkpoints/longformer_large_science.ckpt’ saved [1646843693/1646843693]

--2023-03-23 19:33:30--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/scifact.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.218.209.225, 52.92.176.234, 52.218.220.57, ...
Connecting to scifact.s3.us-west-2

In [3]:
!python /content/multivers/script/get_checkpoint.py fever_sci

--2023-03-23 19:36:08--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/fever_sci.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.92.251.98, 3.5.86.186, 52.92.181.42, ...
Connecting to scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)|52.92.251.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5253768950 (4.9G) [binary/octet-stream]
Saving to: ‘checkpoints/fever_sci.ckpt’

checkpoints/fever_s 100%[===================>]   4.89G  32.0MB/s    in 2m 26s  

2023-03-23 19:38:34 (34.4 MB/s) - ‘checkpoints/fever_sci.ckpt’ saved [5253768950/5253768950]



In [4]:
!mv /content/checkpoints /content/multivers/checkpoints

In [5]:
!gdown https://drive.google.com/uc?id=1uqehbrsMEez0jrUnu_4d12D4BBaHCKcm
!tar -xf data.tar.gz
!rm data.tar.gz
!mv /content/data /content/multivers/data

Downloading...
From: https://drive.google.com/uc?id=1uqehbrsMEez0jrUnu_4d12D4BBaHCKcm
To: /content/data.tar.gz
100% 3.61M/3.61M [00:00<00:00, 171MB/s]


In [6]:
%%capture
!pip install datasets
!pip install codetiming
!pip install jsonlines

In [7]:
from datasets import load_dataset
import pandas as pd
from codetiming import Timer
import jsonlines


In [8]:
ds_orig = load_dataset("climate_fever", split='test')
df = ds_orig.to_pandas()

Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

Dataset climate_fever downloaded and prepared to /root/.cache/huggingface/datasets/climate_fever/default/1.0.1/9091d2d01d327631e4880d42fda9e8d69da1fb59b53b12d68867cee6e5770ba4. Subsequent calls will reuse this data.


In [9]:
for row in df.to_dict('records')[:2]:
  print(row)

{'claim_id': '0', 'claim': 'Global warming is driving polar bears toward extinction', 'claim_label': 0, 'evidences': array([{'evidence_id': 'Extinction risk from global warming:170', 'evidence_label': 2, 'article': 'Extinction risk from global warming', 'evidence': '"Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event".', 'entropy': 0.6931471824645996, 'votes': array(['SUPPORTS', 'NOT_ENOUGH_INFO', None, None, None], dtype=object)},
       {'evidence_id': 'Global warming:14', 'evidence_label': 0, 'article': 'Global warming', 'evidence': 'Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral reefs, mountains, and the Arctic.', 'entropy': 0.0, 'votes': array(['SUPPORTS', 'SUPPORTS', None, None, None], dtype=object)},
       {'evidence_id': 'Global warming:178', 'evidence_label': 2, 'article': 'Global warming', 'evidence': 'Rising temperatures push bees to their physi

In [10]:
cf_label_dict = {'SUPPORTS' : 0, 'REFUTES':1, 'NOT_ENOUGH_INFO' : 2}
mv_label_dict = {0:'SUPPORT', 1: 'CONTRADICT'}

with jsonlines.open('claims.jsonl', 'w') as claims_writer, \
jsonlines.open('corpus.jsonl', 'w') as corpus_writer:
  doc_id = 0
  for row in df.to_dict('records'):
      doc_ids = []
      evidences = row['evidences']
      evidence_dict = {}
      for evidence in evidences:
          evidence_labels = []
          evidence_abstract = {
              'doc_id': doc_id,
              'title': evidence['article'],
              'abstract': [evidence['evidence']]
          }
          evidence_label = evidence['evidence_label']
          # skip NEI
          if evidence_label != 2:
            evidence_labels.append({'sentences': [0],
                                    'label': mv_label_dict[evidence_label]})
            evidence_dict[f"{doc_id}"] = evidence_labels
          corpus_writer.write(evidence_abstract)
          doc_ids.append(doc_id)
          doc_id += 1
      claim_doc = {
          'id': int(row['claim_id']),
          'claim': row['claim'],
          'doc_ids': doc_ids,
          'evidence': evidence_dict
      }
      claims_writer.write(claim_doc)


In [11]:
!head /content/multivers/data/healthver/claims_test.jsonl 

{"id": 1623, "claim": " Saliva testing does depend on standard PCR technology, and it does require some manual labor in order to move it through", "doc_ids": [176], "evidence": {}}
{"id": 1624, "claim": "5G caused the coronavirus outbreak", "doc_ids": [69, 98, 207, 259], "evidence": {"69": [{"sentences": [14], "label": "CONTRADICT"}], "98": [{"sentences": [3], "label": "CONTRADICT"}, {"sentences": [1], "label": "CONTRADICT"}], "207": [{"sentences": [7], "label": "CONTRADICT"}], "259": [{"sentences": [2], "label": "CONTRADICT"}]}}
{"id": 1625, "claim": "A few cases of dogs tested weak positives of Coronavirus and there is evidence they can transmit to humans directly.", "doc_ids": [236], "evidence": {"236": [{"sentences": [1], "label": "CONTRADICT"}]}}
{"id": 1626, "claim": "A headache is a potential symptom of COVID-19.", "doc_ids": [162, 247, 278], "evidence": {"162": [{"sentences": [6], "label": "SUPPORT"}], "247": [{"sentences": [5], "label": "SUPPORT"}], "278": [{"sentences": [2], 

In [12]:
!head claims.jsonl

{"id": 0, "claim": "Global warming is driving polar bears toward extinction", "doc_ids": [0, 1, 2, 3, 4], "evidence": {"1": [{"sentences": [0], "label": "SUPPORT"}], "3": [{"sentences": [0], "label": "SUPPORT"}]}}
{"id": 5, "claim": "The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists", "doc_ids": [5, 6, 7, 8, 9], "evidence": {"5": [{"sentences": [0], "label": "SUPPORT"}], "6": [{"sentences": [0], "label": "SUPPORT"}], "7": [{"sentences": [0], "label": "SUPPORT"}]}}
{"id": 6, "claim": "The polar bear population has been growing.", "doc_ids": [10, 11, 12, 13, 14], "evidence": {"13": [{"sentences": [0], "label": "CONTRADICT"}], "14": [{"sentences": [0], "label": "CONTRADICT"}]}}
{"id": 9, "claim": "Ironic' study finds more CO2 has slightly cooled the planet", "doc_ids": [15, 16, 17, 18, 19], "evidence": {"17": [{"sentences": [0], "label": "CONTRADICT"}]}}
{"id": 10, "claim": "Human additions of CO2 are in the margin of error of curr

In [13]:
!head corpus.jsonl

{"doc_id": 0, "title": "Extinction risk from global warming", "abstract": ["\"Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event\"."]}
{"doc_id": 1, "title": "Global warming", "abstract": ["Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral reefs, mountains, and the Arctic."]}
{"doc_id": 2, "title": "Global warming", "abstract": ["Rising temperatures push bees to their physiological limits, and could cause the extinction of bee populations."]}
{"doc_id": 3, "title": "Habitat destruction", "abstract": ["Rising global temperatures, caused by the greenhouse effect, contribute to habitat destruction, endangering various species, such as the polar bear."]}
{"doc_id": 4, "title": "Polar bear", "abstract": ["\"Bear hunting caught in global warming debate\"."]}
{"doc_id": 5, "title": "Famine", "abstract": ["The current consensus of the scientific community is that the 

In [23]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/scifact.ckpt \
        --batch_size=10 \
        --input_file="/content/claims.jsonl" \
        --corpus_file="/content/corpus.jsonl" \
        --output_file="/content/preds.jsonl"

100% 768/768 [19:58<00:00,  1.56s/it]


In [14]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/scifact.ckpt \
        --batch_size=1 \
        --no_nei \
        --input_file="/content/claims.jsonl" \
        --corpus_file="/content/corpus.jsonl" \
        --output_file="/content/preds_no_nei.jsonl"

Downloading: 100% 803/803 [00:00<00:00, 688kB/s]
Downloading: 100% 1.74G/1.74G [01:01<00:00, 28.2MB/s]
Downloading: 100% 899k/899k [00:00<00:00, 46.1MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 27.7MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 48.8MB/s]
100% 7675/7675 [28:47<00:00,  4.44it/s]


In [15]:
!head /content/preds_no_nei.jsonl 

{"id": 0, "evidence": {"0": {"label": "SUPPORT", "sentences": []}, "1": {"label": "SUPPORT", "sentences": []}, "2": {"label": "SUPPORT", "sentences": []}, "3": {"label": "SUPPORT", "sentences": []}, "4": {"label": "SUPPORT", "sentences": []}}}
{"id": 5, "evidence": {"5": {"label": "CONTRADICT", "sentences": []}, "6": {"label": "CONTRADICT", "sentences": []}, "7": {"label": "CONTRADICT", "sentences": []}, "8": {"label": "CONTRADICT", "sentences": []}, "9": {"label": "CONTRADICT", "sentences": []}}}
{"id": 6, "evidence": {"10": {"label": "SUPPORT", "sentences": []}, "11": {"label": "SUPPORT", "sentences": []}, "12": {"label": "SUPPORT", "sentences": []}, "13": {"label": "SUPPORT", "sentences": []}, "14": {"label": "CONTRADICT", "sentences": []}}}
{"id": 9, "evidence": {"15": {"label": "SUPPORT", "sentences": []}, "16": {"label": "SUPPORT", "sentences": []}, "17": {"label": "SUPPORT", "sentences": []}, "18": {"label": "SUPPORT", "sentences": []}, "19": {"label": "CONTRADICT", "sentences":

In [24]:
!source /content/multivers/bin/activate; python /content/scifact-evaluator/evaluator/eval.py \
        --metrics_output_file=/content/multivers_scifact_eval_climate_fever.jsonl \
        --verbose \
        --labels_file="/content/claims.jsonl" \
        --preds_file="/content/preds.jsonl"

In [16]:
!source /content/multivers/bin/activate; python /content/scifact-evaluator/evaluator/eval.py \
        --metrics_output_file=/content/multivers_scifact_eval_climate_fever_no_nei.jsonl \
        --verbose \
        --labels_file="/content/claims.jsonl" \
        --preds_file="/content/preds_no_nei.jsonl"

In [25]:
!cp /content/multivers_scifact_eval_climate_fever.jsonl /content/drive/MyDrive/omdena_CC_challenge_data/CF_multivers
!cp /content/multivers_scifact_eval_climate_fever_no_nei.jsonl /content/drive/MyDrive/omdena_CC_challenge_data/CF_multivers

# Predict using fever_sci checkpoint

In [18]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/fever_sci.ckpt \
        --batch_size=10 \
        --input_file="/content/claims.jsonl" \
        --corpus_file="/content/corpus.jsonl" \
        --output_file="/content/preds_fever_sci.jsonl"

100% 768/768 [20:04<00:00,  1.57s/it]


In [19]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/fever_sci.ckpt \
        --batch_size=10 \
        --no_nei \
        --input_file="/content/claims.jsonl" \
        --corpus_file="/content/corpus.jsonl" \
        --output_file="/content/preds_fever_sci_no_nei.jsonl"

100% 768/768 [20:02<00:00,  1.57s/it]


# Evaluate using fever_sci checkpoint

In [20]:
!source /content/multivers/bin/activate; python /content/scifact-evaluator/evaluator/eval.py \
        --metrics_output_file=/content/multivers_fever_sci_eval_climate_fever.jsonl \
        --verbose \
        --labels_file="/content/claims.jsonl" \
        --preds_file="/content/preds_fever_sci.jsonl"

In [21]:
!source /content/multivers/bin/activate; python /content/scifact-evaluator/evaluator/eval.py \
        --metrics_output_file=/content/multivers_fever_sci_eval_climate_fever_no_nei.jsonl \
        --verbose \
        --labels_file="/content/claims.jsonl" \
        --preds_file="/content/preds_fever_sci_no_nei.jsonl"

In [22]:
!cp /content/multivers_fever_sci_eval_climate_fever.jsonl /content/drive/MyDrive/omdena_CC_challenge_data/CF_multivers
!cp /content/multivers_fever_sci_eval_climate_fever_no_nei.jsonl /content/drive/MyDrive/omdena_CC_challenge_data/CF_multivers